In [6]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
# hyperparameter

learning_rate = 0.001
training_epochs = 5
batch_size = 100

In [9]:
# dataset

mnist_train = datasets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True
)

mnist_test = datasets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=transforms.ToTensor(),
                          download=True
)

train_loader = DataLoader(dataset=mnist_train,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)

test_loader = DataLoader(dataset=mnist_test,
                         batch_size=batch_size,
                         shuffle=False
                         )

# class

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    
    # 28 x 28 x 1  ->  28 x 28 x 32  ->  14 x 14 x 32

    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),   
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # 14 x 14 x 32  ->  14 x 14 x 64  ->  7 x 7 x 64

    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    # 7 x 7 x 64  ->  7 x 7 x 128  ->  4 x 4 x 128

    self.layer3 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
    )

    # 4 x 4 x 128

    self.fc1 = nn.Linear(4*4*128, 625, bias=True)
    nn.init.xavier_uniform_(self.fc1.weight)
    
    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(p=0.5)
    )

    self.fc2 = nn.Linear(625, 10, bias=True)
    nn.init.xavier_uniform_(self.fc2.weight)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x.view(-1, 4*4*128))

    return self.fc2(x)


model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def train(epoch):
  model.train()
  avg_loss = 0

  for data, target in train_loader:
      data, target = data.to(device), target.to(device)

      optimizer.zero_grad()
      hypothesis = model(data)
      loss = criterion(hypothesis, target)
      loss.backward()
      optimizer.step()

      avg_loss += loss / len(train_loader)

  print('[Epoch: {:>4}] loss = {:>.9}'.format(epoch + 1, avg_loss))


def test():
  model.eval()
  correct = 0

  for data, target in test_loader:
    output = model(data)
    predicted = output.data.max(1, keepdim=True)[1]
    correct += predicted.eq(target.data.view_as(predicted)).sum()  

  data_num = len(test_loader.dataset)  # 데이터 총 건수
  print('\n테스트 데이터에서 예측 정확도: {}/{} ({:.0f}%)\n'.format(correct,
                                                   data_num, 100. * correct / data_num))

if __name__ == '__main__':
  for epoch in range(training_epochs):
    train(epoch)

  test()


[Epoch:    1] loss = 0.194110334
[Epoch:    2] loss = 0.0504854359
[Epoch:    3] loss = 0.0340362564
[Epoch:    4] loss = 0.0255185086
[Epoch:    5] loss = 0.0225109942

테스트 데이터에서 예측 정확도: 9943/10000 (99%)

